In [1]:
import pandas as pd

import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

import seaborn as sns

In [2]:
df_susy = pd.read_csv('heart.csv', engine='c')
df_susy.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [4]:
X=df_susy[['age', 'sex', 'cp', 'trestbps',  'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']]
# Labels
y=df_susy['target']

# Split dataset into training set and test set
# 70% training and 30% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, 
                                                          random_state=100,
                                                          shuffle=True,
                                                          stratify=y)
print("Pronto!")

Pronto!


In [5]:
#Extend from sklearn.base BaseEstimator, ClassifierMixin
# import threading
# from joblib import Parallel, delayed
class BaseEnginnering(object):
    def __init__(self):
        pass
    
    def get_subset(self, X, y, columns):
        from pandas import DataFrame

        #if not isinstance(self, X, DataFrame):
            #raise TypeError('Expected value should descend from pandas.core.frame.DataFrame')
            
        df_tmp = X.copy()
        df_tmp.insert(loc=(len(df_tmp.columns)), column='target', value=y)
        
        #df_subset = (df_tmp[df_tmp.columns in columns], df_tmp['target'])
        df_subset = (df_tmp.loc[:, columns], df_tmp['target'])
        
        return df_subset
    
    def arrangement_features(self, features: list,  n_selected: int) -> list:
        from itertools import combinations
        permsList = list(combinations(features, r=n_selected))
        
        return permsList

print('Pronto!')

Pronto!


In [7]:
class ClassifierEnginneringForest(BaseEnginnering):
    
    def __init__(self):
        self.criterion = 'entropy'
        self.splitter='best'
        self.max_depth=None
        self.min_samples_split=2
        self.min_samples_leaf=1
        self.min_weight_fraction_leaf=0
        self.max_features=None
        self.random_state = 200
        self.max_leaf_nodes=None
        #min_impurity_decrease=0,
        #min_impurity_split=1e-7,
        self.class_weight=None
        self.presort=False
        
    def make_base_estimator(self):
        clf = DecisionTreeClassifier(self.criterion)
        return clf
    
    def make_lote_base_estimator(self):
        pass
    
print('Pronto!')

Pronto!


In [8]:
class EnginneringForest(ClassifierEnginneringForest):

    def __init__(self, select_features):
        # Global variables
        self.estimators_ = []
        self.select_features_ = select_features
        self.group_features = []
        self.vector_predict = []
        super().__init__()
        
    def build(self, features_set):
        # Aqui vai o código para criar a floresta
        # cria o vetor dos subconjuntos
        # cria as instâncias das arvores
        self.group_features = self.arrangement_features(features=features_set, n_selected=self.select_features_)
        #self.estimators_ = [self.make_base_estimator() for item_set in self.group_features]
        for i in self.group_features:
            self.estimators_.append(self.make_base_estimator())
            
    def voting(self):
        final_predict = []
        for i in range(len(self.vector_predict[0])):
            column_predict = []
            for j in range(len(self.estimators_)):
                column_predict.append(self.vector_predict[j][i])
            if column_predict.count(1) > column_predict.count(0):
                final_predict.append(1)
            else:
                final_predict.append(0)
        return final_predict
                
        
    def fit(self, X, y):
         # Determine output settings
        n_samples, self.n_features_ = X.shape
        name_features = X.columns
        
        # Cria a floresta
        self.build(features_set=name_features)
        
        # Treina as arvores individualmente
        for subset_feature, estimators in zip(self.group_features, self.estimators_):
            subset_xdata, subset_ydata = self.get_subset(X, y, subset_feature)
            estimators = estimators.fit(subset_xdata, subset_ydata)
        
        # Treina a floresta
        #for item_set, clf in zip(self.group_features, self.estimators_):
         #   subset_xdata, subset_ydata = self.get_subset(X, y, item_set)
         #   clf = clf.fit(subset_xdata, subset_ydata)
        #for subset_feature in subsets_features:
         #   subset_xdata, subset_ydata = self.get_subset(X, y, subset_feature)
         #   clf = self.build()
         #   clf = clf.fit(subset_xdata, subset_ydata)
         #   self.estimators_.append(clf)
            
    def predict(self, X):
        for subset_feature, estimators in zip(self.group_features, self.estimators_):
            subset_test = X.loc[:, subset_feature]
            self.vector_predict.append(estimators.predict(subset_test))
        return self.voting()
            

print('Pronto!')
# usar map para percorrer o vetor de arvores

Pronto!


In [9]:
model = EnginneringForest(select_features=7)

In [10]:
model.fit(X_train, y_train)

In [11]:
y_pred = model.predict(X_test)

In [12]:
mac = metrics.accuracy_score(y_test, y_pred)
mcr = classification_report(y_test,y_pred)
mas = accuracy_score(y_test, y_pred)
mcm = confusion_matrix(y_test,y_pred)

print("Accuracy:", mac)

print("\nClassifcation Report")
print(mcr)  

print("\nAccuracy Score")
print(mas)

print("\nConfusion Matrix")
print(mcm)

Accuracy: 0.8461538461538461

Classifcation Report
              precision    recall  f1-score   support

           0       0.86      0.78      0.82        41
           1       0.83      0.90      0.87        50

    accuracy                           0.85        91
   macro avg       0.85      0.84      0.84        91
weighted avg       0.85      0.85      0.85        91


Accuracy Score
0.8461538461538461

Confusion Matrix
[[32  9]
 [ 5 45]]


In [14]:
model_rf = RandomForestClassifier(n_estimators= 100,
                                  criterion='entropy',
                                  max_features = 'auto',
                                  max_depth = None,
                                  min_samples_split = 2,
                                  min_samples_leaf = 1,
                                  min_weight_fraction_leaf = 0,
                                  max_leaf_nodes = None,
                                  min_impurity_decrease = 0,
                                  bootstrap = True,
                                  oob_score = True,
                                  n_jobs = -1,
                                  random_state = 100,
                                  verbose = 1,
                                  warm_start = False,
                                  class_weight = None)
mrf_fit = model_rf.fit(X_train, y_train)
y_pred = model_rf.predict(X_test)
mac = metrics.accuracy_score(y_test, y_pred)
mcr = classification_report(y_test,y_pred)
mas = accuracy_score(y_test, y_pred)
mcm = confusion_matrix(y_test,y_pred)

print("Accuracy:", mac)

print("\nClassifcation Report")
print(mcr)  

print("\nAccuracy Score")
print(mas)

print("\nConfusion Matrix")
print(mcm)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.0s finished


Accuracy: 0.8461538461538461

Classifcation Report
              precision    recall  f1-score   support

           0       0.86      0.78      0.82        41
           1       0.83      0.90      0.87        50

    accuracy                           0.85        91
   macro avg       0.85      0.84      0.84        91
weighted avg       0.85      0.85      0.85        91


Accuracy Score
0.8461538461538461

Confusion Matrix
[[32  9]
 [ 5 45]]


In [57]:
model.estimators_[0].predict(X_test.loc[:, model.group_features[0]])

array([0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1,
       0, 1, 0], dtype=int64)

In [58]:
model.estimators_[1].predict(X_test.loc[:, model.group_features[1]])

array([0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0,
       1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 0], dtype=int64)

In [60]:
len(model.predict(X_test))

91

In [31]:
model.vector_predict[0]

array([0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1,
       0, 1, 0], dtype=int64)

In [32]:
len(model.vector_predict[0])

91

In [ ]:
#X_test.loc[['age', 'sex', 'cp']].iloc[0]
X_test.loc[214, ['age', 'sex', 'cp']]

In [ ]:
X_test.index

In [ ]:
len(model.estimators_)

In [ ]:
for p in model.estimators_:
    print(p.feature_importances_)

def predict(self, X):
        # Só pode fazer a predição com os atributos usado para treinar a arvore
        # Senão dá erro
        #x_length = X.shape[0]
        for item in X.index:
            x_predict = []
            for subset_feature, estimators in zip(self.group_features, self.estimators_):
                subset_test = X.loc[item, subset_feature]
                x_predict.append(estimators.predict(subset_test))
            self.vector_predict.append(x_predict)
            x_predict.clear()